In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%aimport bs4, ebooklib, spacy, html.parser

from bs4 import BeautifulSoup
from ebooklib import epub

from html.parser import HTMLParser

In [ ]:
from clean_parse_tag import extract_tags_by_attributes, get_all_tag_names, get_all_attribute_values 
from clean_parse_tag import remove_all_tags_with_attribute, remove_tags_with_attribute, remove_attributes, remove_empty_tags, unwrap_redundant_tags
from clean_parse_tag import remove_tag_whitespace, normalize_newlines, clean_text

In [ ]:
from clean_parse_tag import set_working_directory, get_text_from_file, write_text_to_file 

In [ ]:
TH_book_path = "../books/private_books/Transformation_and_Healing-Thich_Nhat_Hanh.epub"
LiA_book_path = "../books/private_books/Love_in_Action-Thich_Nhat_Hanh.epub"
TH_epub_book = epub.read_epub(TH_book_path)
LiA_epub_book = epub.read_epub(LiA_book_path)

In [ ]:
soups = [BeautifulSoup(item.get_body_content(), 'html.parser') for item in TH_epub_book.get_items_of_media_type('application/xhtml+xml')]

In [ ]:
len(soups)

In [ ]:
print(soups[18].prettify())

In [ ]:
soups[18].find_all('br')

In [ ]:
get_all_tag_names(soups[18])

In [ ]:
tagset = set()
for i, soup in enumerate(soups):
    tags = get_all_tag_names(soup)
    for tag in tags:
        tagset.add(tag)
print(tagset)

In [ ]:
for i, soup in enumerate(soups):
    print(i, soup.find_all('sup'))

In [ ]:
def generate_tag_dict(soup_list):
    tag_dict = {tag: {} for tag in tagset}
    for soup in soup_list:
        for tag in tagset:
            attr_dict = get_all_attribute_values(soup, tag)
            for attr, value_set in attr_dict.items():
                if attr in tag_dict[tag]:
                    tag_dict[tag][attr].update(value_set)
                else:
                    tag_dict[tag][attr] = value_set
    return tag_dict

In [ ]:
generate_tag_dict(soups)

In [ ]:
print(soups[1].prettify())

In [ ]:
print(soups[18].prettify())

In [ ]:
for soup in soups:
    remove_attributes(soup, "style", r"margin*")

In [ ]:
print(soups[1].prettify())

In [ ]:
for soup in soups:
    remove_attributes(soup, "class", r"calibre*")

In [ ]:
for soup in soups:
    remove_attributes(soup, "id", r"filepos.*")

In [ ]:
for soup in soups:
    remove_attributes(soup, 'class', 'mbp_*')

In [ ]:
for soup in soups:
    remove_attributes(soup, 'id', "calibre_pb*")

In [ ]:
for soup in soups:
    remove_attributes(soup, 'href', "index_*")

In [ ]:
for soup in soups:
    remove_empty_tags(soup, ['p', 'span', 'div', 'blockquote', 'a'])

In [ ]:
print(soups[1].prettify())

In [ ]:
for soup in soups:
   unwrap_redundant_tags(soup, ['a', 'span'])

In [ ]:
print(soups[1].prettify())

In [ ]:
generate_tag_dict(soups)

## Begin code to explore matching html data with text data:

key idea is to use an html parser and event stream to parse the html tree.

In [ ]:
set_working_directory('../books/private_books')
text = get_text_from_file('transformation_and_healing_tnh.txt')

In [ ]:
print(text)

In [ ]:
text = normalize_newlines(text, 2)

In [ ]:
text = clean_text(text)

In [ ]:
write_text_to_file('test_th_output.txt', text)

In [ ]:
print(soups[18].prettify())

In [ ]:
cleaned_html = clean_text(str(soups[1])) 

In [ ]:
cleaned_html = remove_tag_whitespace(cleaned_html)

In [ ]:
print(cleaned_html)

In [ ]:
def contains_non_breaking_space(text):
    """
    Check if a string contains the non-breaking space character (Unicode \xa0).

    Parameters:
    - text (str): The string to check.

    Returns:
    - bool: True if \xa0 is found, False otherwise.
    """
    return "\xa0" in text

In [ ]:
contains_non_breaking_space(text)

In [ ]:
len(text)

In [ ]:
contains_non_breaking_space(cleaned_html)

In [ ]:
print(text[:1200])

In [ ]:
text_input = text[:307].split('\n')

In [ ]:
text_input

In [ ]:
class HTMLTextMatcher(HTMLParser):
    def __init__(self, text_data):
        super().__init__()
        self.text_iterator = iter(text_data)  # Iterator over the text lines or words
        self.current_text_segment = next(self.text_iterator, "").strip()
        self.mismatches = []

    def handle_starttag(self, tag, attrs):
        # We may want to log start tags, but they typically don't have matching text
        print(f"Start tag: <{tag}>")

    def handle_endtag(self, tag):
        print(f"End tag: </{tag}>")

    def handle_data(self, data):
        html_text = data.strip()
        
        if html_text:
            print(f"HTML Text: {html_text}")

            # Attempt to match with the current segment of the text document
            if self.current_text_segment == html_text:
                print(f"Matched: {html_text}")
                # Move to the next text segment after a match
                self.current_text_segment = next(self.text_iterator, "").strip()
            else:
                # Log the mismatch for later review
                print(f"Mismatch: HTML='{html_text}' vs Text='{self.current_text_segment}'")
                self.mismatches.append((html_text, self.current_text_segment))
                # Advance to the next segment even on mismatch to keep up with the stream
                self.current_text_segment = next(self.text_iterator, "").strip()

    def get_mismatches(self):
        return self.mismatches

# # Sample usage
# html_data = """
# <body>
#  <p><span class="bold">Table of Contents</span></p>
#  <p><span class="italic"><span class="underline">Title Page</span></span></p>
#  <p><span class="italic"><span class="underline">A NOTE ON THE TEXT</span></span></p>
#  <p><span class="italic"><span class="underline">Introduction</span></span></p>
#  <p><span class="italic"><span class="underline">Sutra on the Four Establishments of Mindfulness</span></span></p>
# </body>
# """
# text_data = [
#     "Transformation and Healing: Sutra on the Four Establishments of Mindfulness",
#     "Table of Contents",
#     "Title Page",
#     "A NOTE ON THE TEXT",
#     "Introduction",
#     "Sutra on the Four Establishments of Mindfulness",
# ]

# # Instantiate the matcher with text data
# parser = HTMLTextMatcher(text_data)
# # Feed the cleaned HTML data
# parser.feed(html_data)

# # Review mismatches, if any
# print("Mismatches found:", parser.get_mismatches())

In [ ]:
from html.parser import HTMLParser

class HTMLPlainTextMatcher(HTMLParser):
    def __init__(self):
        super().__init__()
        self.plain_text = ""         # Accumulated plain text without tags
        self.tag_map = []            # Stores tag position data
        self.current_index = 0       # Tracks position in the plain text

    def handle_starttag(self, tag, attrs):
        # Convert attributes to a string format, e.g., <tag attr1="val1" attr2="val2">
        attr_str = " ".join(f'{key}="{value}"' for key, value in attrs)
        full_tag = f"<{tag} {attr_str}>" if attr_str else f"<{tag}>"

        # Log start of the tag with the current index
        self.tag_map.append((self.current_index, None, full_tag))

    def handle_endtag(self, tag):
        # Find last open tag in tag_map for this tag type and close it with the current index
        for i in range(len(self.tag_map) - 1, -1, -1):
            if self.tag_map[i][1] is None and self.tag_map[i][2].startswith(f"<{tag}"):
                self.tag_map[i] = (self.tag_map[i][0], self.current_index, self.tag_map[i][2])  # Update with end_index
                break
         # Close <p> tags with double newline for paragraph separation
        if tag in ['p', 'blockquote']:
            self.plain_text += "\n\n"  # Add double newline after closing <p>
            self.current_index += 2  # Update index to account for newlines

    def handle_data(self, data):
        # Strip the single unit of text and add double newline for readability
        # stripped_data = data.strip() # don't strip data; save whitespace to match text.
        if data:
            self.plain_text += data
            self.current_index += len(data)  # Update position after adding text and newlines

    def handle_startendtag(self, tag, attrs):
        # Capture full tag with attributes, if present, for logging
        attr_str = " ".join(f'{key}="{value}"' for key, value in attrs)
        full_tag = f"<{tag} {attr_str} />" if attr_str else f"<{tag} />"
        self.tag_map.append((self.current_index, full_tag))

        # For <br> add newlines for readability
        if tag == "br":
            self.plain_text += "\n\n"  # Single newline for line break
            self.current_index += 2  # Update index for newline

    def get_plain_text(self):
        return self.plain_text

    def get_tag_map(self):
        return self.tag_map

In [ ]:
parser = HTMLPlainTextMatcher()

In [ ]:
parser.feed(cleaned_html)

In [ ]:
print(parser.plain_text)

In [ ]:
parser.tag_map

In [ ]:
import difflib

# Example of using SequenceMatcher for diff alignment
html_test_text = parser.plain_text
plain_text_test = text

html_data = html_test_text #.split()
text_data = plain_text_test #.split()

# Create a SequenceMatcher object
matcher = difflib.SequenceMatcher(None, html_data, text_data)

# Get matching blocks
matches = matcher.get_matching_blocks()
for match in matches:
    print("Match found:", match)

In [ ]:
text_data[77:77+609]

In [ ]:
# Accessing the start index and length of a matching block
for match in matches:
    start = match.a  # Starting index in html_data
    length = match.size  # Number of matching words

    # Retrieve the matching words
    matching_words = html_data[start:start + length]
    print("Matching words:", matching_words)

In [ ]:
len(html_data), len(text_data)